# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [91]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [92]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,299.18,65,94,7.20,CK,1699495464
1,1,tourlaville,49.6383,-1.5664,280.83,87,75,4.63,FR,1699495713
2,2,stanley,54.8680,-1.6985,278.43,92,75,1.54,GB,1699495198
3,3,port-aux-francais,-49.3500,70.2167,278.91,94,100,16.40,TF,1699495730
4,4,grytviken,-54.2811,-36.5092,276.66,77,100,1.08,GS,1699495738


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [93]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City", alpha = 0.6, size = "Humidity", tiles ='OSM')

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [94]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[(city_data_df['Humidity']<35) & (city_data_df['Max Temp'] <= 299)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,adrar,20.5022,-10.0711,292.32,30,0,4.45,MR,1699496047
47,47,korla,41.7597,86.1469,280.44,25,27,2.28,CN,1699496120
50,50,crane,31.3974,-102.3501,298.77,26,93,0.78,US,1699496144
104,104,lobujya,27.9500,86.8167,264.39,27,4,0.80,NP,1699496603
117,117,tamanrasset,22.7850,5.5228,292.07,16,51,3.77,DZ,1699496721
155,155,xinyuan,37.3000,99.0000,273.62,17,0,4.10,CN,1699497053
183,183,baicheng,45.6167,122.8167,271.04,33,0,4.70,CN,1699497418
205,205,santa ana,33.7456,-117.8678,296.29,21,0,3.58,US,1699497634
211,211,pahrump,36.2083,-115.9839,287.50,20,0,2.68,US,1699497777
234,234,turpan,42.9333,89.1667,284.63,17,46,0.28,CN,1699497973


### Step 3: Create a new DataFrame called `hotel_df`.

In [95]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
39,39,adrar,20.5022,-10.0711,292.32,30,0,4.45,MR,1699496047,
47,47,korla,41.7597,86.1469,280.44,25,27,2.28,CN,1699496120,
50,50,crane,31.3974,-102.3501,298.77,26,93,0.78,US,1699496144,
104,104,lobujya,27.9500,86.8167,264.39,27,4,0.80,NP,1699496603,
117,117,tamanrasset,22.7850,5.5228,292.07,16,51,3.77,DZ,1699496721,
155,155,xinyuan,37.3000,99.0000,273.62,17,0,4.10,CN,1699497053,
183,183,baicheng,45.6167,122.8167,271.04,33,0,4.70,CN,1699497418,
205,205,santa ana,33.7456,-117.8678,296.29,21,0,3.58,US,1699497634,
211,211,pahrump,36.2083,-115.9839,287.50,20,0,2.68,US,1699497777,
234,234,turpan,42.9333,89.1667,284.63,17,46,0.28,CN,1699497973,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [97]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel", "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adrar - nearest hotel: No hotel found
korla - nearest hotel: Silver Star Hotel
crane - nearest hotel: No hotel found
lobujya - nearest hotel: Himalaya Lodge
tamanrasset - nearest hotel: فندق الأمان
xinyuan - nearest hotel: No hotel found
baicheng - nearest hotel: 白城鶴翔大酒店
santa ana - nearest hotel: Holiday Inn Express
pahrump - nearest hotel: Holiday Inn Express & Suites
turpan - nearest hotel: Oriental
hami - nearest hotel: Хами
taoudenni - nearest hotel: No hotel found
aioun - nearest hotel: فندق العيون
alamogordo - nearest hotel: Days Inn Alamogordo
kidal - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
sultanah - nearest hotel: فندق جلنار
bechar - nearest hotel: Hôtel Madania
qarqaraly - nearest hotel: Каркаралинск


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
39,39,adrar,20.5022,-10.0711,292.32,30,0,4.45,MR,1699496047,No hotel found
47,47,korla,41.7597,86.1469,280.44,25,27,2.28,CN,1699496120,Silver Star Hotel
50,50,crane,31.3974,-102.3501,298.77,26,93,0.78,US,1699496144,No hotel found
104,104,lobujya,27.9500,86.8167,264.39,27,4,0.80,NP,1699496603,Himalaya Lodge
117,117,tamanrasset,22.7850,5.5228,292.07,16,51,3.77,DZ,1699496721,فندق الأمان
155,155,xinyuan,37.3000,99.0000,273.62,17,0,4.10,CN,1699497053,No hotel found
183,183,baicheng,45.6167,122.8167,271.04,33,0,4.70,CN,1699497418,白城鶴翔大酒店
205,205,santa ana,33.7456,-117.8678,296.29,21,0,3.58,US,1699497634,Holiday Inn Express
211,211,pahrump,36.2083,-115.9839,287.50,20,0,2.68,US,1699497777,Holiday Inn Express & Suites
234,234,turpan,42.9333,89.1667,284.63,17,46,0.28,CN,1699497973,Oriental


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [98]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
39,39,adrar,20.5022,-10.0711,292.32,30,0,4.45,MR,1699496047,No hotel found
47,47,korla,41.7597,86.1469,280.44,25,27,2.28,CN,1699496120,Silver Star Hotel
50,50,crane,31.3974,-102.3501,298.77,26,93,0.78,US,1699496144,No hotel found
104,104,lobujya,27.9500,86.8167,264.39,27,4,0.80,NP,1699496603,Himalaya Lodge
117,117,tamanrasset,22.7850,5.5228,292.07,16,51,3.77,DZ,1699496721,فندق الأمان
155,155,xinyuan,37.3000,99.0000,273.62,17,0,4.10,CN,1699497053,No hotel found
183,183,baicheng,45.6167,122.8167,271.04,33,0,4.70,CN,1699497418,白城鶴翔大酒店
205,205,santa ana,33.7456,-117.8678,296.29,21,0,3.58,US,1699497634,Holiday Inn Express
211,211,pahrump,36.2083,-115.9839,287.50,20,0,2.68,US,1699497777,Holiday Inn Express & Suites
234,234,turpan,42.9333,89.1667,284.63,17,46,0.28,CN,1699497973,Oriental


In [99]:
%%capture --no-display
#Excluding No hotel found rows
hotel_df = hotel_df[~(hotel_df['Hotel Name'] == 'No hotel found')]
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng", "Lat", 
                                geo = True, color = "City", alpha = 0.6, 
                                size = hotel_df["Humidity"]*4, tiles ='OSM', 
                                xlim=(-120,120), ylim=(-50, 70), 
                                frame_height=450, frame_width=800, 
                                hover_cols=['Hotel Name', 'Country'])
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Hotel Name,Country)